```bash
 iopaint start --model=lama --device=mps --port=8080 --enable-remove-bg  --enable-interactive-seg --interactive-seg-device=mps --enable-gfpgan  --enable-realesrgan --enable-anime-seg --enable-restoreformer
```

- enable-restoreformer : 저화질 -> 고화질
- enable-gfpgan : 얼굴 복원
- enable-realesrgan : 높은 해상도로 업스케일
- enable-anime-seg : 에니메이션 세그멘테이션


In [ ]:
import cv2
import numpy as np
import time
from PIL import Image

image, alpha_channel, infos, ext = decode_base64_to_image(req.image)
mask, _, _, _ = decode_base64_to_image(req.mask, gray=True)
print(f"image ext: {ext}")

mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
if image.shape[:2] != mask.shape[:2]:
    raise HTTPException(
        400,
        detail=f"Image size({image.shape[:2]}) and mask size({mask.shape[:2]}) not match.",
    )

start = time.time()
rgb_np_img = self.model_manager(image, mask, req)
logger.info(f"process time: {(time.time() - start) * 1000:.2f}ms")
torch_gc()

rgb_np_img = cv2.cvtColor(rgb_np_img.astype(np.uint8), cv2.COLOR_BGR2RGB)
rgb_res = concat_alpha_channel(rgb_np_img, alpha_channel)

res_img_bytes = pil_to_bytes(
    Image.fromarray(rgb_res),
    ext=ext,
    quality=self.config.quality,
    infos=infos,
)

asyncio.run(self.sio.emit("diffusion_finish"))

return Response(
    content=res_img_bytes,
    media_type=f"image/{ext}",
    headers={"X-Seed": str(req.sd_seed)},
)